In [174]:
include("inversion_ext.jl")

 Activating environment at `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
  Resolving package versions...
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Manifest.toml`


f_to_c!

In [175]:
N=10
M=1
χ = [-1.0; -1.0; -5.0]
p = (χ, N, M)
u = zeros(N*M)
u0 = zeros(N*M)

con = view(reshape(u,(M,N)),:,1:5)
flu = view(reshape(u,(M,N)),:,6:10)

flu[1,:] = [-10.0, 0.2, 1.1, 1.5, 1.2]
con[1,:] = [0.1366314976448222, 0.07009306769467444, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818]

con0 = view(reshape(u0,(M,N)),:,1:5)
flu0 = view(reshape(u0,(M,N)),:,6:10)

flu0[1,:] = [-10.0, 0.2, 1.1, 1.5, 1.2]
#con0[1,:] = [0.1366314976448222, 0.07009306769467444, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818]

println(u0)
println(u)
println(f_to_c!(u0,p) - u)
println(u0)

[0.0, 0.0, 0.0, 0.0, 0.0, -10.0, 0.2, 1.1, 1.5, 1.2]
[0.1366314976448222, 0.07009306769467444, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818, -10.0, 0.2, 1.1, 1.5, 1.2]
[0.0, -1.3877787807814457e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.1366314976448222, 0.07009306769467442, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818, -10.0, 0.2, 1.1, 1.5, 1.2]


In [176]:
tol = 10^(-16)
iter_max = 40
χ = [-1.0; -1.0; -5.0]
M=1
N=10
p = (χ, tol, iter_max, N, M)

u = zeros(N*M)
u0 = zeros(N*M)

con = view(reshape(u,(M,N)),:,1:5)
flu = view(reshape(u,(M,N)),:,6:10)

flu[1,:] = [-10.0, 0.2, 1.1, 1.5, 1.2]
con[1,:] = [0.1366314976448222, 0.07009306769467444, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818]

con0 = view(reshape(u0,(M,N)),:,1:5)
flu0 = view(reshape(u0,(M,N)),:,6:10)

flu0[1,:] = [-10.0, 0.2, 1.1, 1.5, 1.2] + 0.01 .*rand(N÷2)
con0[1,:] = [0.1366314976448222, 0.07009306769467444, 0.06115332989597844, 0.07178418128379448, 0.04927907295689818]

5-element Array{Float64,1}:
 0.1366314976448222
 0.07009306769467444
 0.06115332989597844
 0.07178418128379448
 0.04927907295689818

In [178]:
@time c_to_f!(u0, p) - u

  0.000073 seconds (34 allocations: 2.156 KiB)


10-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  4.806112841038157e-9
  2.941994980965035e-10
 -2.7242426092755068e-9
 -2.2971502477986405e-9
 -1.4237011569662172e-10